In [1]:
import bs4
import json
import requests as rq
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import json
import time

In [2]:

#for today date
def today():
    return datetime.today().strftime('%Y-%m-%d')
#for now time
def now_time():
    return time.strftime("%H:%M:%S", time.localtime(time.time()))

#for getting kakao auth token
def kakao_connect(access_token):

    kakao_url="https://kauth.kakao.com/oauth/token"

    data={
        "grant_type" : "authorization_code",
        "client_id" : "62f53fd06d840ed70de64e7337840e6b",
        "redirect_url" : "https://localhost.com",
        "code" : access_token
    }
    response=rq.post(kakao_url,data=data)
    tokens=response.json()
    with open("kakao_token.json", "w") as fp:
        json.dump(tokens, fp)


#for getting URL
def driver():
    my_driver=webdriver.Chrome('chromedriver.exe')
    my_driver.get("http://consensus.hankyung.com/apps.analysis/analysis.list")
    return my_driver


#for activating Driver_URL
def driver_active(keywords): 
    date=[]
    hrink=[]
    title=[]
    my_driver=driver()
    for keyword in keywords:
        search=my_driver.find_element_by_id('search_text')
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)
        dates=[x.text for x in my_driver.find_elements_by_css_selector('td.first.txt_number')]
        hrinks=[x.get_attribute('href') for x in my_driver.find_elements_by_css_selector('div.dv_input>a')]
        titles=[x.text for x in my_driver.find_elements_by_css_selector('td.text_l')]
        date.append(dates)
        hrink.append(hrinks)
        title.append(titles)
        my_driver.back()
        time.sleep(3)
    return {'dates':date,'hrinks':hrink,'titles':title}


#for getting fitted today data
def get_customized_info(dictionary):
    url=[]
    title=[]
    for i in range(len(dictionary['dates'])):
        for j in range(len(dictionary['dates'][i])):
            if  dictionary['dates'][i][j]==today():
                url.append(dictionary['hrinks'][i][j])
                title.append(re.sub(r'[|\?*<:>/.]','',dictionary['titles'][i][j]))
    return {'url':url,'title':title}

#sending to kakao for text
def send_to_kakao(text):  
    kakao_host="https://kapi.kakao.com/v2/api/talk/memo/default/send"
    headers = {"Authorization" : "Bearer "+tokens['access_token']}
    post={"object_type" : "text",
          "text" : text,
          "link" : {"web_url" : "http://consensus.hankyung.com/apps.analysis/analysis.list","mobile_web_url" : "http://consensus.hankyung.com/apps.analysis/analysis.list"}
          }
    data = {"template_object" : json.dumps(post)}
    
    return rq.post(kakao_host,headers=headers,data=data)


#sending data to my mobile kakao
def sending(tag,send_list):
    rink=tag['url']
    title=tag['title']
    for i in range(len(rink)):
        if (title[i] in send_list):
            print("보낸적 있음")
            continue
        else:
            send_to_kakao(title[i] +'\n'+ rink[i])
            send_list.append(title[i])
            time.sleep(5)
    return send_list

In [3]:
#access_token='9_s6QqcshppbC7uvARnX3zpHmuqNLBL8ZLT35jGcD5UBWgTteOeple0-vfejNt3WpQGdRQopb9UAAAFzk0bpSA'
with open("kakao_token.json", "r") as fp:
    tokens=json.load(fp)

In [4]:
keywords=['삼성','반도체']
send_list=[]
info=driver_active(keywords)
tag=get_customized_info(info)
sending(tag,send_list)

['삼성물산(028260)우려대비 선방',
 '반도체-Meritz Overnight Tech',
 '반도체-TSMC, 인텔로부터 6nm 위탁생산 주문 수주']

In [ ]:
sending(tag,send_list)

In [ ]:
#send_list=[]
# now=now_time()
# value = time.strftime("09:20:00")
# close = time.strftime("12:00:00")
# while value == now:
#         while True:
#             keywords=['sk','samsung']
#             info=driver_active(keywords)
#             tag=get_customized_info(info)
#             sending(tag,send_list)
#             time.sleep(60*10)
#             if now_time==close:
#                 break